In [1]:
import warnings 
warnings.filterwarnings('ignore')
import shutil
import os 
from skimage import io
import numpy as np
import matplotlib.pyplot as plt 
import random
from PIL import Image
from torch.autograd import Variable 
from sklearn.metrics import accuracy_score
import pandas as pd
import copy
import shutil


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torchvision

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
# Прописываем путь к файлу с архивом
zip_file = '/content/drive/MyDrive/CNN_LSTM/datasets.zip'

# Распаковываем архив
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()


### Подгрузка данных без трансформа и экстрактора  

In [5]:
class Make_Dataset(Dataset):

    def __init__(self, root_dir, ):
        
        
        # Дир с сетом
        self.root_dir = root_dir
        
        # Название файлов с драками
        self.fight_names = os.listdir(self.root_dir + 'fight')
        # Название файлов с без драк
        self.nonfight_names = os.listdir(self.root_dir + 'noFight')
        self.all_dirs = [self.root_dir + 'fight/' + i for i in self.fight_names] +\
        [self.root_dir + 'noFight/' + i for i in self.nonfight_names] 
        
        random.shuffle(self.all_dirs) 
        
        self.my_transforms = transforms.Compose([
                                transforms.Resize((720, 1280)),
                                
                                ])

    def __len__(self):
        
        #Размер сета
        return len(os.listdir(self.root_dir + 'fight'))+len(os.listdir(self.root_dir + 'noFight'))

    
    # Обработка последовательног набора фотограф
    def __work_with_one_dir(self,idx,):
        
        images = []
        
            
        # Путь с набором фоток для 1 видео
        dirs = self.all_dirs[idx]
            
        # Полные пути фоток с драками для 1 видео 
        img_names = [os.path.join(dirs + '/'+i) for i in os.listdir(dirs)]
            
        for i in img_names:
            # Грузим
            
            images.append(np.array(self.my_transforms(Image.open(i))))
            #images.append(np.array(Image.open(i)))

            
        
        if 'noFight' in self.all_dirs[idx]:
            
            sample = { 'data':np.array(images),'lbl': 0}
        
        else:
            
            sample = { 'data':np.array(images),'lbl': 1}
        
       
       
        return sample
        
        
    # Получаем 1 набор данных                                                         
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
                idx = idx.tolist()
                
        
        sample = self.__work_with_one_dir(idx,)
            
            
       

        

        return sample

### BI-LSTM class

In [6]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, bidirectional =True) #lstm
        #self.fc_1 =  nn.Linear(self.hidden_size*2*self.num_layers, 1024)
        self.fc_1 =  nn.Linear(self.hidden_size*2, 1024) #fully connected 1
        self.fc = nn.Linear(1024, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        hn = hn.view(-1, self.hidden_size*2*self.num_layers) #reshaping the data for Dense layer next
        #out = self.relu(hn)
        out=self.relu(output[:,-1,:])
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

### Для сохран тенз

In [7]:
def save_tensor(dataloader, name):
    



    data = []
    lbls = []
    for i in dataloader:
    
        data = data + list(np.array(i['data']))
        lbls = lbls + list(np.array(i['lbl']))
        
    torch.save(torch.tensor(data), name + '_data.pt')
    torch.save(torch.tensor(lbls), name + '_lbl.pt')



### Класс для extracted_dataset

In [8]:
class Make_extracted_Dataset(Dataset):

    def __init__(self, my_dict, ):
        
        self.my_dict = my_dict
       

    def __len__(self):
        
        #Размер сета
        return len(self.my_dict['lbl'])

        
        
    # Получаем 1 набор данных                                                         
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
                idx = idx.tolist()
                
        
        sample = {'data':self.my_dict['data'][idx], 'lbl': self.my_dict['lbl'][idx]}
            
            
       

        

        return sample

### Функции для обучения

In [9]:
def creat_extracted_dataset(extraktor, dataset,size,number_of_frames):
    
    lbls = []
    inputs = []
    for i in dataset:
        
        extracketed_input = [extraktor(i['data'][k].reshape((number_of_frames,3,720,1280)).float())\
                             .detach().numpy() for k in range(len(i['data']))]
        
        extracketed_input = torch.tensor(extracketed_input).reshape((i['data'].shape[0],number_of_frames,size))
        
        
        
        
        
        
        lbls = lbls +list(i['lbl'])
        inputs = inputs + list(np.array(extracketed_input))
    
    return  np.array(inputs),np.array(lbls),
        




def train( model, criterion,  optimizer, train_dataloader, val_dataloader, num_epochs ):
    
    
    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)
    
    best_metric = 0
    best_accuracy_test = 0
    best_accuracy_train = 0

    loss_train_hist = []
    loss_val_hist = []
    accuracy_train_hist = []
    accuracy_val_hist = []

    for epoch in range(num_epochs):
        model.train()
        for i in train_dataloader:
            
            
            
        
            lbl = Variable(torch.nn.functional.one_hot(i['lbl'],num_classes=2).to(device)) 
            data = Variable(i['data'].to(device))
        
        
            outputs = model.forward(data) #forward pass
            optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
            # obtain the loss functio
        
        
        
            loss = criterion(outputs, lbl.float())


        
        
 
            loss.backward() #calculates the loss of the loss function
 
            optimizer.step() #improve from loss, i.e backprop
    
    
    
        model.eval()
        preds = []
        lbls = []
        for i in val_dataloader:
            
            
            
        
            lbl = list(np.array(i['lbl']))
            lbls = lbls + lbl
    
            outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
            preds = preds + outputs
        
        
        
    
     
        loss = float(criterion(torch.nn.functional.one_hot(torch.tensor(lbls),num_classes=2).float(),
                           torch.nn.functional.one_hot(torch.tensor(preds), num_classes=2).float()))
        test_acc = accuracy_score(lbls, preds)

        loss_val_hist.append(loss)
        accuracy_val_hist.append(test_acc)
    
        
    
        #print(f'Точность на валедации на эпохе {epoch+1} = {acc}')
        #print(f'Loss на валедации на эпохе {epoch+1} = {loss}')
    
        if test_acc > best_accuracy_test:
        
            best_model_wts = copy.deepcopy(model.state_dict())
            best_accuracy_test = test_acc
    
        preds = []
        lbls = []
        for i in train_dataloader_extracted:
        
        
            
            lbl = list(np.array(i['lbl']))
            lbls = lbls + lbl
    
            outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
            preds = preds + outputs
        
        
        
    
     
        loss = float(criterion(torch.nn.functional.one_hot(torch.tensor(lbls),num_classes=2).float(),
                           torch.nn.functional.one_hot(torch.tensor(preds), num_classes=2).float()))
    
        train_acc = accuracy_score(lbls, preds)
    
        loss_train_hist.append(loss)
        accuracy_train_hist.append(train_acc)

        if train_acc>best_accuracy_train:

          best_accuracy_train = train_acc


        if test_acc == best_accuracy_test:

          if  train_acc>=best_accuracy_train:

            best_model_wts = copy.deepcopy(model.state_dict())


        #print(f'Точность на трейне на эпохе {epoch+1} = {acc}')
        #print(f'Loss на трейне на эпохе {epoch+1} = {loss}')

    return best_model_wts



### Обучение

In [10]:
extraktor = torchvision.models.resnet152(pretrained=True)
extraktor = torch.nn.Sequential(*(list(extraktor.children())[:-1]))

  # Ключ имя экстрактора, значение по ключу сам экстрактор и значение выходных признаков векторов
dict_of_extraktors = {'resnet152' : (extraktor, 2048)}
extraktor = torchvision.models.vgg16(pretrained=True)
dict_of_extraktors['vgg16'] = (extraktor, 1000)
extraktor = torchvision.models.alexnet(pretrained=True)
dict_of_extraktors['alexnet'] = (extraktor, 1000)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [11]:
train_dataset = Make_Dataset('/content/datasets/10/frames/train/')

test_dataloader = Make_Dataset('/content/datasets/10/frames/test/')

train_dataloader = DataLoader(train_dataset, batch_size=4,
                        shuffle=True, num_workers=0,)


test_dataloader = DataLoader(test_dataloader, batch_size=4,
                        shuffle=True, num_workers=0,)

In [12]:
%%time
number_of_frames = 10 
for z in list(dict_of_extraktors.keys()):
  print(z)
  data, lbls = creat_extracted_dataset(dict_of_extraktors[z][0], test_dataloader,dict_of_extraktors[z][1],number_of_frames)
  test_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':data, 'lbl':lbls}), batch_size=4,
                                         shuffle=False, num_workers=0,)
  
  
  data, lbls = creat_extracted_dataset(dict_of_extraktors[z][0], train_dataloader,dict_of_extraktors[z][1],number_of_frames)
  train_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':data, 'lbl':lbls}), batch_size=4,
                                          shuffle=False, num_workers=0,)
  
  save_tensor(test_dataloader_extracted, f'test_{z}_number_of_frames_{number_of_frames}')
  save_tensor(train_dataloader_extracted, f'train_{z}_number_of_frames_{number_of_frames}')

  shutil.copyfile(f'/content/test_{z}_number_of_frames_{number_of_frames}_data.pt',f'/content/drive/MyDrive/CNN_LSTM/test_{z}_number_of_frames_{number_of_frames}_data.pt')
  shutil.copyfile(f'/content/test_{z}_number_of_frames_{number_of_frames}_lbl.pt',f'/content/drive/MyDrive/CNN_LSTM/test_{z}_number_of_frames_{number_of_frames}_lbl.pt')

  shutil.copyfile(f'/content/train_{z}_number_of_frames_{number_of_frames}_data.pt',f'/content/drive/MyDrive/CNN_LSTM/train_{z}_number_of_frames_{number_of_frames}_data.pt')
  shutil.copyfile(f'/content/train_{z}_number_of_frames_{number_of_frames}_lbl.pt',f'/content/drive/MyDrive/CNN_LSTM/train_{z}_number_of_frames_{number_of_frames}_lbl.pt')

  


resnet152
vgg16
alexnet
CPU times: user 11h 28min 22s, sys: 2h 57min 6s, total: 14h 25min 29s
Wall time: 3h 51min 52s


In [13]:
train_dataset = Make_Dataset('/content/datasets/5/frames/train/')

test_dataloader = Make_Dataset('/content/datasets/5/frames/test/')

train_dataloader = DataLoader(train_dataset, batch_size=4,
                        shuffle=True, num_workers=0,)


test_dataloader = DataLoader(test_dataloader, batch_size=4,
                        shuffle=True, num_workers=0,)

In [14]:
%%time
number_of_frames = 5 
for z in list(dict_of_extraktors.keys()):
  print(z)
  data, lbls = creat_extracted_dataset(dict_of_extraktors[z][0], test_dataloader,dict_of_extraktors[z][1],number_of_frames)
  test_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':data, 'lbl':lbls}), batch_size=4,
                                         shuffle=False, num_workers=0,)
  
  
  data, lbls = creat_extracted_dataset(dict_of_extraktors[z][0], train_dataloader,dict_of_extraktors[z][1],number_of_frames)
  train_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':data, 'lbl':lbls}), batch_size=4,
                                          shuffle=False, num_workers=0,)
  
  save_tensor(test_dataloader_extracted, f'test_{z}_number_of_frames_{number_of_frames}')
  save_tensor(train_dataloader_extracted, f'train_{z}_number_of_frames_{number_of_frames}')

  shutil.copyfile(f'/content/test_{z}_number_of_frames_{number_of_frames}_data.pt',f'/content/drive/MyDrive/CNN_LSTM/test_{z}_number_of_frames_{number_of_frames}_data.pt')
  shutil.copyfile(f'/content/test_{z}_number_of_frames_{number_of_frames}_lbl.pt',f'/content/drive/MyDrive/CNN_LSTM/test_{z}_number_of_frames_{number_of_frames}_lbl.pt')

  shutil.copyfile(f'/content/train_{z}_number_of_frames_{number_of_frames}_data.pt',f'/content/drive/MyDrive/CNN_LSTM/train_{z}_number_of_frames_{number_of_frames}_data.pt')
  shutil.copyfile(f'/content/train_{z}_number_of_frames_{number_of_frames}_lbl.pt',f'/content/drive/MyDrive/CNN_LSTM/train_{z}_number_of_frames_{number_of_frames}_lbl.pt')

  


resnet152
vgg16
alexnet
CPU times: user 5h 56min 14s, sys: 1h 32min 37s, total: 7h 28min 51s
Wall time: 2h 5s
